Database and table setup

In [18]:
CREATE DATABASE BookstoreDB;

USE BookstoreDB;

: Msg 1801, Level 16, State 3, Line 1
Database 'BookstoreDB' already exists. Choose a different database name.

Total execution time: 00:00:00.006

In [4]:
CREATE TABLE Authors (
    AuthorID INT PRIMARY KEY IDENTITY(1,1),
    FirstName NVARCHAR(100),
    LastName NVARCHAR(100)
);

CREATE TABLE Books (
    BookID INT PRIMARY KEY IDENTITY (1,1),
    Title NVARCHAR(255),
    AuthorID INT FOREIGN KEY REFERENCES Authors(AuthorID),
    Price DECIMAL(10,2),
    Stock INT
);

: Msg 2714, Level 16, State 6, Line 1
There is already an object named 'Authors' in the database.

Total execution time: 00:00:00.029

In [7]:
--Insert data 
INSERT INTO Authors (FirstName, LastName)
VALUES ('J.K.', 'Rowling'), 
    ('George', 'Orwell'), 
    ('J.R.R.', 'Tolkien'),
    ('Agatha', 'Christie'),
    ('Isaac', 'Asimov'),
    ('Stephen', 'King');

INSERT INTO Books (Title, AuthorID, Price, Stock)
VALUES ('Harry Potter', 1, 20.99, 10),
       ('1984', 2, 15.99, 5),
       ('The Hobbit', 3, 12.99, 8),
       ('Murder on the Orient Express', 4, 9.99, 7),
       ('And Then There Were None', 4, 8.99, 10),
       ('Foundation', 5, 11.99, 12),
       ('I, Robot', 5, 10.99, 14),
       ('The Shining', 6, 15.99, 6),
       ('It', 6, 17.99, 5);

(6 rows affected)

(9 rows affected)

Total execution time: 00:00:00.041

In [10]:
-- List all books and their authors, first name abbreviated to initial
SELECT 
    B.Title, 
    SUBSTRING(A.FirstName, 1, 1) AS Initial, 
    A.LastName, 
    B.Price
FROM 
    Books B
JOIN 
    Authors A ON B.AuthorID = A.AuthorID;  


(9 rows affected)

(1 row affected)

Total execution time: 00:00:00.048

Title,Initial,LastName,Price
Harry Potter,J,Rowling,20.99
1984,G,Orwell,15.99
The Hobbit,J,Tolkien,12.99
Murder on the Orient Express,A,Christie,9.99
And Then There Were None,A,Christie,8.99
Foundation,I,Asimov,11.99
"I, Robot",I,Asimov,10.99
The Shining,S,King,15.99
It,S,King,17.99


In [13]:
--Change a price
UPDATE Books
SET Price = 13.99 -- The new price
WHERE Title = '1984';


(1 row affected)

Total execution time: 00:00:00.009

In [17]:
--Identify books priced under $15 to be put in 'Bargain' section
SELECT 
    Title, 
    Price,
    CASE 
        WHEN Price < 15.00 THEN 'Bargain'
        ELSE 'Regular'
    END AS Section
FROM Books
WHERE Price < 15.00

(6 rows affected)

Total execution time: 00:00:00.011

Title,Price,Section
1984,13.99,Bargain
The Hobbit,12.99,Bargain
Murder on the Orient Express,9.99,Bargain
And Then There Were None,8.99,Bargain
Foundation,11.99,Bargain
"I, Robot",10.99,Bargain


In [ ]:
--Update stock to reflect sales
UPDATE Books
SET Stock = Stock - 1
WHERE BookID = 1;

In [29]:
--Stored procedure to decrease stock when book is sold
CREATE PROCEDURE SellBook
@BookID INT,
AS
BEGIN 
    UPDATE Books 
    SET Stock = Stock - 1
    WHERE BookID = @BookID
--For handling negative stock
IF (SELECT Stock FROM Books WHERE BookID = @BookID) < 0
    BEGIN
        RAISERROR('Stock cannot be negative', 16, 1);
    END
END;
GO

: Msg 2714, Level 16, State 3, Procedure BookSold, Line 2
There is already an object named 'BookSold' in the database.

Total execution time: 00:00:00.017

In [36]:
--Mistake made when creating SellBook procedure, modify to fix
ALTER PROCEDURE SellBook
    @BookID INT,
    @QuantitySold INT
AS
BEGIN
    UPDATE Books
    SET Stock = Stock - @QuantitySold
    WHERE BookID = @BookID;
    
    -- Optional: Check for negative stock and handle it.
    IF (SELECT Stock FROM Books WHERE BookID = @BookID) < 0
    BEGIN
        RAISERROR('Stock cannot be negative', 16, 1);
        -- Optionally, rollback any transaction if you have one
    END
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.008

In [38]:
--Check inventory of Book with BookID = 3
SELECT BookID, Title, Stock
FROM Books
WHERE BookID = 3

-- Assume we sold 2 copies of the book with BookID = 3
EXEC  @BookID = 3, @QuantitySold = 2;

-- Fetch the updated stock for the book with BookID = 3
SELECT Title, Stock 
FROM Books 
WHERE BookID = 3;


(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.012

BookID,Title,Stock
3,The Hobbit,6


Title,Stock
The Hobbit,4


In [39]:
--Check current inventory counts
SELECT B.Title, 
A.FirstName, 
A.LastName, 
B.Stock
FROM 
    Books B
JOIN
    Authors A
ON 
    B.AuthorID = A.AuthorID


(9 rows affected)

Total execution time: 00:00:00.036

Title,FirstName,LastName,Stock
Harry Potter,J.K.,Rowling,10
1984,George,Orwell,5
The Hobbit,J.R.R.,Tolkien,4
Murder on the Orient Express,Agatha,Christie,7
And Then There Were None,Agatha,Christie,10
Foundation,Isaac,Asimov,12
"I, Robot",Isaac,Asimov,14
The Shining,Stephen,King,6
It,Stephen,King,5
